In [194]:
import pandas as pd
from pathlib import Path
from ConfigSpace import ConfigurationSpace
from deepcave import Recorder, Objective
from deepcave.runs.converters.deepcave import DeepCAVERun
from deepcave.plugins.hyperparameter.importances import Importances

In [195]:
cs_branin = ConfigurationSpace({"x1": (-5.0, 10.0), "x2": (0.0, 15.0)})
cs_mlp = ConfigurationSpace(
    {
        "n_layer": (1, 5),
        "n_neurons": (8, 1024),
        "activation": ["logistic", "tanh", "relu"],
        "solver": ["lbfgs", "sgd", "adam"],
        "batch_size": (30, 300),
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "learning_rate_init": (0.0001, 1.0),
    }
)
cs_sac = ConfigurationSpace(
    {
        "algorithm.model_kwargs.learning_rate": (0.000001, 0.01),
        "algorithm.model_kwargs.batch_size": [64, 128, 256, 512],
        "algorithm.model_kwargs.gradient_steps": (1, 10),
        "algorithm.model_kwargs.learning_starts": (10, 10000),
        "algorithm.model_kwargs.buffer_size": (5e3, 5e7),
    }
)
configspaces = {"branin": cs_branin, "mlp": cs_mlp, "sac": cs_sac}

In [196]:
def data_to_deepcave(data, target_function):
    configspace = configspaces[target_function]
    default = configspace.get_default_configuration()
    accuracy = Objective(
        "reward_mean", lower=min(data["performance"]), upper=max(data["performance"]), optimize="upper"
    )
    save_path = "deepcave_logs"

    with Recorder(configspace, objectives=[accuracy], save_path=save_path) as r:
        for index, d in data.iterrows():
            current_hps = d
            config = default
            for c in current_hps.keys():
                key = c.split(".")[-1]
                if key in config.keys():
                    try:
                        config[key] = current_hps[c]
                    except:
                        config[key] = default[c]
            seed = 0
            if "seed" in d.keys():
                config["seed"] = d["seed"]

            r.start(config=config, budget=d["budget"], seed=seed)
            r.end(config=config, costs=d["performance"], budget=d["budget"], seed=seed)

In [197]:
def get_importances(path, target_function, method="local"):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_ids = run.get_budget_ids()

    # Instantiate the plugin
    plugin = Importances()

    inputs = plugin.generate_inputs(
        objective_id1=objective_id,
        objective_id2=None,
        hyperparameter_names=configspaces[target_function].get_hyperparameter_names(),
        budget_ids=budget_ids,
        method=method,
        n_hps=10,
        n_trees=10,
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    # Finally, you can load the figure. Here, the filter variables play a role.
    # Alternatively: Use the matplotlib output (`load_mpl_outputs`) if available.
    figure = plugin.load_outputs(run, inputs, outputs)  # plotly.go figure
    return figure

In [198]:
# TODO: add your result path here
result_path = "tmp/2024-09-04/15-21-47"

# Load runhistory
config_data = pd.read_csv(f"{result_path}/runhistory.csv", index_col=False)

# Generate deepcave logs from runhistory
data_to_deepcave(config_data, "branin")

In [199]:
# Now we can use deepcave to analyze the results - e.g. for hyperparameter importance:
get_importances("deepcave_logs/run_1", "branin")

/var/folders/yc/_2wjgdhd4px91396bq5wm7kr0000gn/T/ipykernel_78484/869544127.py:13: DeprecationWarning:

Please use `list(space.keys())`

